In [6]:
import sys
import os
# Add the directory containing db.py to the system path
script_dir = "C:\\Users\\User\\Projects\\requires-rewrite-gui"
script_dir2 = "C:\\Users\\oriro\\Projects\\requires-rewrite-gui"
sys.path.append(script_dir)
sys.path.append(script_dir2)
# Now you can import db.py
from db import *
from collections import Counter

def count_completed_dialogs_perc(json_data):

    def is_rewrite_empty(rewrite_data):
        if rewrite_data["requires_rewrite"] == None:
            return True
        elif rewrite_data["enough_context"] == None:
            return True
        else:
            return False


    num_of_dialogs = len(json_data)
    counter = 0
    for dialog_data in json_data.values():

        for key, value in dialog_data.items():
            if key.isdigit():
                if is_rewrite_empty(value):
                    return (counter / num_of_dialogs)
        
        counter += 1

    return 1.0

def count_completed_dialogs(json_data):

    counter = -1
    
    
    for dialog_value in json_data.values():
        counter += 1
        for turn_key, turn_value in dialog_value["dialog"].items():
            if turn_key == "0":
                continue
            
            else:
                if "requires_rewrite" in turn_value:
                    if turn_value["requires_rewrite"] == None:
                        return counter
                    else:
                        continue
                        
    return counter

def print_annotation(json_data):
    counter = 1
    for dialog_id, dialog_data in json_data.items():
        print(f"{counter} | Dialog ID: {dialog_id}")
        counter += 1
        for turn_id, turn_data in dialog_data["dialog"].items():
            if turn_id == "0":
                continue
            print(f"\tTurn ID: {turn_id}")
            print(f"\tRequires Rewrite: {turn_data['requires_rewrite']}")
            print(f"\tEnough Context: {turn_data['enough_context']}")
            print()


def topn(json_data, n=10):
    # return the top n items in json data
    top_n = {}
    counter = 0
    for dialog_id, dialog_data in json_data.items():
        top_n[dialog_id] = dialog_data
        counter += 1
        if counter == n:
            break
    return top_n
    
    
def give_score(json_template, annotations):
    annotators_scores = {"ori": 0, "AfikK": 0, "lieli": 0}
    for dialog_id, dialog_data in json_template.items():
        for turn_id, turn_data in dialog_data.items():
            if turn_id.isdigit():
                
                annotators_annotations = {}
                for annotator, annotator_data in annotations.items():
                    rr = annotator_data[dialog_id][turn_id]["requires_rewrite"]
                    ec = annotator_data[dialog_id][turn_id]["enough_context"]
                    if rr == None or ec == None:
                        return annotators_scores
                    annotators_annotations[annotator] = (rr, ec)
                
                rr_max = Counter([rr for rr, ec in annotators_annotations.values()]).most_common(1)[0][0]
                
                ec_max = Counter([ec for rr, ec in annotators_annotations.values()]).most_common(1)[0][0]

                rr_counter = 0
                ec_counter = 0
                for rr, ec in annotators_annotations.values():
                    if rr == rr_max:
                        rr_counter += 1
                    if ec == ec_max:
                        ec_counter += 1
                
                for annotator, (rr, ec) in annotators_annotations.items():
                    
                    if rr != rr_max and rr_counter < 3:
                        annotators_scores[annotator] += 1
                    if ec != ec_max and ec_counter < 3:
                        annotators_scores[annotator] += 1      
    return annotators_scores     

In [4]:
annotators_json_data = retrieve_annotations_by_file_id("asi-23_4")


ori
AfikK
lieli
here i am


In [11]:
print_annotation(topn(annotators_json_data["test111"], 20))

1 | Dialog ID: QReCC-Train_4784
	Turn ID: 1
	Requires Rewrite: 1
	Enough Context: 1

2 | Dialog ID: QReCC-Train_63
	Turn ID: 1
	Requires Rewrite: 0
	Enough Context: 1

3 | Dialog ID: QReCC-Train_1883
	Turn ID: 1
	Requires Rewrite: 1
	Enough Context: 0

4 | Dialog ID: QReCC-Train_7455
	Turn ID: 1
	Requires Rewrite: 1
	Enough Context: 1

5 | Dialog ID: QReCC-Train_1354
	Turn ID: 1
	Requires Rewrite: 0
	Enough Context: 1

	Turn ID: 2
	Requires Rewrite: 1
	Enough Context: 0

	Turn ID: 3
	Requires Rewrite: None
	Enough Context: None

6 | Dialog ID: QReCC-Train_5399
	Turn ID: 1
	Requires Rewrite: 1
	Enough Context: 1

7 | Dialog ID: QReCC-Train_6870
	Turn ID: 1
	Requires Rewrite: 0
	Enough Context: 1

	Turn ID: 2
	Requires Rewrite: 1
	Enough Context: 0

8 | Dialog ID: QReCC-Train_4326
	Turn ID: 1
	Requires Rewrite: 0
	Enough Context: 1

9 | Dialog ID: QReCC-Train_6510
	Turn ID: 1
	Requires Rewrite: 0
	Enough Context: 1

10 | Dialog ID: QReCC-Train_619
	Turn ID: 1
	Requires Rewrite: 0
	Enough

In [ ]:
annotators_json_data = retrieve_annotations_by_file_id("asi-23_4")

for annotator, data in annotators_json_data.items():
    print(f"Annotator: {annotator}")
    print(f"Completed dialogs: {count_completed_dialogs(data)}")
    
    print("\n\n")

ori
AfikK
lieli
Annotator: ori
Completed dialogs: 1441



Annotator: AfikK
Completed dialogs: 1441



Annotator: lieli
Completed dialogs: 1441



Annotator: yaelk
Completed dialogs: 4



Annotator: yaelk1
Completed dialogs: 4



Annotator: test9
Completed dialogs: 4



Annotator: test10
Completed dialogs: 4



Annotator: test11
Completed dialogs: 4



Annotator: test111
Completed dialogs: 4



Annotator: test69
Completed dialogs: 4



Annotator: test1
Completed dialogs: 4



Annotator: test
Completed dialogs: 4





In [ ]:
list(annotators_json_data["test10"].items())[:10]

[('QReCC-Train_4784',
  {'number_of_turns': 2,
   'annotator_id': 'test10',
   'dialog': {'0': {'turn_num': 0,
     'sample_id': 'QReCC-Train_4784_1',
     'original_question': 'What happened with Kate Tsui in 2007?',
     'answer': 'Kate Tsui earned the Most Improved Female Artist award from the TVB Anniversary Awards 2007.'},
    '1': {'turn_num': 1,
     'sample_id': 'QReCC-Train_4784_2',
     'original_question': 'What happened in 2008?',
     'answer': 'Kate Tsui earned her a Top 5 position in the Best Supporting Actress category of the TVB Anniversary Awards 2008.',
     'requires_rewrite': 0,
     'enough_context': 1}}}),
 ('QReCC-Train_63',
  {'number_of_turns': 2,
   'annotator_id': 'test10',
   'dialog': {'0': {'turn_num': 0,
     'sample_id': 'QReCC-Train_63_1',
     'original_question': 'Where was Yui Horie born?',
     'answer': 'Yui Horie was born in Tokyo'},
    '1': {'turn_num': 1,
     'sample_id': 'QReCC-Train_63_2',
     'original_question': 'What happened in the ear